# LangChain and LangGraph examples

In this guide, we will see how to integrate Literal AI in a LangGraph workflow.

- [Necessary imports](#imports)
- [Define tools](#define-available-tools)
- [Agent logic](#agent-logic)
- [Run agent](#run-agent)

<a id="imports"></a>
## Necessary imports

Make sure to define the `LITERAL_API_KEY`, `OPENAI_API_KEY` and `TAVILY_API_KEY` in your `.env`.

In [4]:
from typing import Annotated
from langchain_core.messages import HumanMessage

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from dotenv import load_dotenv
from langchain.schema.runnable.config import RunnableConfig
import os
from literalai import LiteralClient

# Check environment variables

In [5]:
load_dotenv()
print(os.getenv("LITERAL_API_KEY"))
print(os.getenv("TAVILY_API_KEY"))
print(os.getenv("OPENAI_API_KEY"))

lsk_Q6RcSpjtJnfl16z8H4lcthmwBqffEVdO4cbUk4Y5o
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
sk-Pp5mufXH5qVjfvaqyjWa5-RzHLxRlbMgU9m8aVJc7ET3BlbkFJo19FHiCFkUQ0rD2U3MQIyluY3P2788Mvrx9xpvLn0A


<a id="define-available-tools"></a>
## Define available tools

We will use Tavily as a search tool. `tools` is a list of available tools  (here, we only have one tool, the TavilySearchResults tool).

In [6]:
load_dotenv()
literal_client = LiteralClient()

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# Define the tool (TavilySearchResults tool to search the web)
tool = TavilySearchResults(max_results=2, k=2)
tools = [tool]

# Define the LLM (chatGPT 4o-mini)
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

<a id="agent-logic"></a>
## Agent logic

For the agent logic, we simply repeat the following pattern (max. 5 times):
- ask the user question to the LLM, making the tools available
- execute tools if LLM asks for it, otherwise return message

In [8]:
# Define the chatbot logic
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


# Add the chatbot node to the graph
graph_builder.add_node("chatbot", chatbot)

# Add the tool node to the graph
tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile()


Adding a node to a graph that has already been compiled. This will not be reflected in the compiled graph.


ValueError: Node `chatbot` already present.

<a id="run-agent"></a>
## Run agent against a question

The agent has a pre-set user question (What is the weather in Paris?). It is run in a separate thread to log the output in literal, and then prints the result.

In [9]:
# wait for user input and then run the graph
with literal_client.thread(name="Weather in Paris") as thread:
    user_input = "What is the weather in Paris?"
    cb = literal_client.langchain_callback()
    res = graph.invoke({"messages": [HumanMessage(content=user_input)]}, config=RunnableConfig(callbacks=[cb]))
    print(res["messages"][-1].content)

The current weather in Paris is as follows:

- **Temperature**: 13.2°C (55.8°F)
- **Condition**: Overcast
- **Wind**: Northeast at 13.2 mph (21.2 kph)
- **Humidity**: 94%
- **Visibility**: 10 km
- **Pressure**: 1026 mb

You can find more detailed weather information [here](https://www.weatherapi.com/). 

As for the upcoming days, Paris is expected to experience cooler weather with temperatures gradually decreasing as September progresses, averaging around 15°C by the end of the month, along with light to moderate rainfall.
